In [40]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score #кросс-валидация

import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов
from hyperopt import space_eval

import optuna


%matplotlib inline
plt.style.use('seaborn')

In [41]:
data = pd.read_csv('data/_train_sem09 (1).csv')

#Создаем матрицу наблюдений X и вектор ответов y
X = data.drop(['Activity'], axis=1)
y = data['Activity']

#задаем random_state
random_state = 42

#Разделяем выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=random_state, test_size=0.2)

## Логистическая регрессия

Смотрим метрику базовой модели

In [42]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=random_state)

#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

y_train_pred = log_reg.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.89
f1_score на тестовом наборе: 0.78


c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
params = {'penalty': ['l2', 'none'] ,
        'solver': ['lbfgs', 'sag', 'saga']
    }

Подбираем параметры с помощью GridSearchCV

In [44]:
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state), 
    param_grid=params, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train)

y_train_pred = grid_search.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 6.16 s
Wall time: 34.6 s
f1_score на тренировочном наборе: 0.88
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'saga'}


c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Подбираем параметры с помощью RandomizedSearchCV

In [45]:
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state), 
    param_distributions=params, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train)

y_train_pred = random_search.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: total: 6.41 s
Wall time: 33.4 s
f1_score на тренировочном наборе: 0.88
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l2'}


c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Подбираем параметры с помощью Hyperopt

In [52]:
params_hyperopt = {'penalty': hp.choice(label='penalty', options=['l2', 'none']),
       'solver' : hp.choice(label='solver', options=['lbfgs', 'saga', 'sag'])
      }

In [53]:
#функция для минимизации, она должна принимать словарь значений гиперпараметров и возвращать значение целевой функции
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [54]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=params_hyperopt, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=10, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 10%|█         | 1/10 [00:03<00:28,  3.13s/trial, best loss: -0.7507880100918504]

c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 20%|██        | 2/10 [00:16<01:12,  9.07s/trial, best loss: -0.7771163161747902]

...

 80%|████████  | 8/10 [01:16<00:23, 11.53s/trial, best loss: -0.7793179449996321]

c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 90%|█████████ | 9/10 [01:19<00:08,  8.79s/trial, best loss: -0.7793179449996321]

c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 10/10 [01:33<00:00,  9.32s/trial, best loss: -0.7793179449996321]
CPU times: total: 49.1 s
Wall time: 1min 33s


Смотрим метрику на полученных значениях

In [55]:
best_params = space_eval(params_hyperopt, best)

model = linear_model.LogisticRegression(
    random_state=random_state, 
    penalty=best_params['penalty'],
    solver=best_params['solver']
)

model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров {}".format(best_params))

f1_score на обучающем наборе: 0.88
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'saga'}


c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Optuna

In [49]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l2', 'none'])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'sag', 'saga'])

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))
  
  #используем кросс-валидацию
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [50]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=10)

[I 2023-01-10 01:40:47,212] A new study created in memory with name: LogisticRegression
c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2023-01-10 01:40:50,272] Trial 0 finished with value: 0.7734882711782042 and parameters: {'penalty': 'l2', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.7734882711782042.
c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITER

CPU times: total: 1min 48s
Wall time: 4min 18s


In [51]:
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)

print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

print("Наилучшие значения гиперпараметров {}".format(study.best_params))

f1_score на обучающем наборе: 0.78
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'saga'}


c:\Users\illus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Результаты:
- Базовая модель - на обучающем наборе метрика f1 = 0.89, на тестовом 0.78
- GridSearchCV - на обучающем наборе метрика f1 = 0.88, на тестовом 0.78
- RandomizedSearchCV - на обучающем наборе метрика f1 = 0.88, на тестовом 0.78
- Hyperopt - на обучающем наборе метрика f1 = 0.88, на тестовом 0.78
- Optuna - на обучающем наборе метрика f1 = 0.78, на тестовом 0.78

## Случайный лес

Смотрим метрику базовой модели

In [27]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=random_state)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print("f1_score на обучающем наборе: {:.2f}".format(metrics.f1_score(y_train, y_train_pred)))

y_test_pred = rf.predict(X_test)
print("f1_score на тестовом наборе: {:.2f}".format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 1.00
f1_score на тестовом наборе: 0.80


Задаем пространство гиперпараметров

In [28]:
param_rf = {'n_estimators': list(range(100, 300, 25)),
              'min_samples_leaf': [3, 5, 7, 9],
              'max_depth': list(np.linspace(15, 40, 5, dtype=int))
              }

Подбираем параметры с помощью GridSearchCV

In [24]:
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state), 
    param_grid=param_rf, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_train, y_train)

y_train_pred = grid_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: total: 4.56 s
Wall time: 6min 23s
f1_score на обучающем наборе: 0.97
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'max_depth': 15, 'min_samples_leaf': 3, 'n_estimators': 175}


Подбираем параметры с помощью RandomizedSearchCV

In [30]:
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state), 
    param_distributions=param_rf, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train)

y_train_pred = random_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 2.44 s
Wall time: 31.9 s
f1_score на обучающем наборе: 0.97
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'n_estimators': 200, 'min_samples_leaf': 3, 'max_depth': 27}


Подбираем параметры с помощью Hyperopt

In [31]:
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 25),
       'max_depth' : hp.quniform('max_depth', 15, 40, 5),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 9, 2)
      }

In [32]:
#функция для минимизации, она должна принимать словарь значений гиперпараметров и возвращать значение целевой функции
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [33]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )

100%|██████████| 20/20 [01:41<00:00,  5.05s/trial, best loss: -0.8140523013373183]
CPU times: total: 43.4 s
Wall time: 1min 41s


In [34]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров {}".format(best))

f1_score на обучающем наборе: 0.93
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров {'max_depth': 35.0, 'min_samples_leaf': 6.0, 'n_estimators': 275.0}


Подбираем параметры с помощью Optuna

In [35]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))
  
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [36]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-01-10 01:07:22,415] A new study created in memory with name: RandomForestClassifier
[I 2023-01-10 01:07:26,223] Trial 0 finished with value: 0.8043868192338881 and parameters: {'n_estimators': 129, 'max_depth': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8043868192338881.
[I 2023-01-10 01:07:29,651] Trial 1 finished with value: 0.7986916009737228 and parameters: {'n_estimators': 121, 'max_depth': 26, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.8043868192338881.
[I 2023-01-10 01:07:34,100] Trial 2 finished with value: 0.8001311715922792 and parameters: {'n_estimators': 157, 'max_depth': 13, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.8043868192338881.
[I 2023-01-10 01:07:38,750] Trial 3 finished with value: 0.8052793717739366 and parameters: {'n_estimators': 167, 'max_depth': 24, 'min_samples_leaf': 7}. Best is trial 3 with value: 0.8052793717739366.
[I 2023-01-10 01:07:43,751] Trial 4 finished with value: 0.8087077268923697 and parameters: {'n_

CPU times: total: 38.3 s
Wall time: 1min 42s


In [37]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

f1_score на обучающем наборе: 0.82
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров {'n_estimators': 180, 'max_depth': 29, 'min_samples_leaf': 2}


Результаты:
- Базовая модель - на обучающем наборе метрика f1 = 1, на тестовом 0.8
- GridSearchCV - на обучающем наборе метрика f1 = 0.97, на тестовом 0.8
- RandomizedSearchCV - на обучающем наборе метрика f1 = 0.97, на тестовом 0.81
- Hyperopt - на обучающем наборе метрика f1 = 0.93, на тестовом 0.8
- Optuna - на обучающем наборе метрика f1 = 0.82, на тестовом 0.8